In [3]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("groq_api")

In [2]:
groq_model = init_chat_model("groq:qwen/qwen3-32b")

In [3]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="Title of the movie")
    year: int = Field(description="This year the movie was released")
    director: str = Field(description="Director of the movie")
    ratings:int = Field(description="The movies rating out of 10")

model_with_structure = groq_model.with_structured_output(Movie,include_ratings=True)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000024BBFEED400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024BBFEEE120>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'Title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'Director of the movie', 'type': 'string'}, 'ratings': {'description': 'The movies rating out of 10', 'type': 'integer'}}, 'required': ['title', 'year', 'direct

In [4]:
#Model Calling with structure:
response = model_with_structure.invoke("Tell about Game of thrones")
response

Movie(title='Game of Thrones (Pilot)', year=2011, director='Tim Van Patten', ratings=9)

In [5]:
response

Movie(title='Game of Thrones (Pilot)', year=2011, director='Tim Van Patten', ratings=9)

# For Pydantic Structure we can use via,
1) with_structured_output()
2) importing and defining the pydantic parser.

# Nested Structure:

In [7]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name:str
    role:str
class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None, description = "Budget in USD Millions")

groq_with_nested_structure = groq_model.with_structured_output(MovieDetails)
nested_response = groq_with_nested_structure.invoke("Provide the details about Game Of thrones")

MovieDetails(title='Game of Thrones', year=2010, cast=[Actor(name='Sean Bean', role='Ned Stark'), Actor(name='Mark Addy', role='Robert Baratheon'), Actor(name='Lena Headey', role='Cersei Lannister')], genres=['Fantasy', 'Adventure'], budget=20.0)

In [13]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(
    pydantic_object = MovieDetails,
)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Actor": {"properties": {"name": {"title": "Name", "type": "string"}, "role": {"title": "Role", "type": "string"}}, "required": ["name", "role"], "title": "Actor", "type": "object"}}, "properties": {"title": {"title": "Title", "type": "string"}, "year": {"title": "Year", "type": "integer"}, "cast": {"items": {"$ref": "#/$defs/Actor"}, "title": "Cast", "type": "array"}, "genres": {"items": {"type": "string"}, "title": "Genres", "type": "array"}, "budget": {"anyOf": [{"type": "number"}, {"type": "null"}], "default": null, "des

# TypedDict:

In [15]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A Movie with details"""
    title: Annotated[str, ..., "the title of movie"]
    year: Annotated[int, ..., "the year of movie released"]
    director: Annotated[str, ..., "the director of movie"]
    ratings: Annotated[int, ..., "the rating of movie"]

# This is same as Pydantic but the input's are not validated in runtime

In [16]:
groq_model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

# DataClasses

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate

# 1. Initialize Embeddings (Local)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Your Knowledge System (QA Pairs)
# In a real app, you might load this from a JSON or CSV
qa_data = [
    {"q": "What is the return policy?", "a": "You can return items within 30 days."},
    {"q": "How do I track my order?", "a": "Use the tracking link sent to your email."},
    {"q": "Do you ship internationally?", "a": "Yes, we ship to over 50 countries."}
]

# Convert to LangChain Document format
from langchain_core.documents import Document
docs = [Document(page_content=f"Question: {item['q']}\nAnswer: {item['a']}") for item in qa_data]

# 3. Create FAISS Vector Database
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

In [16]:
# Set your API Key
from langchain_core.runnables import RunnableLambda
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1 # Low temperature for factual Q&A
)
# 4. Define the Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# 5. Create the Retrieval Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(query):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    docs = retriever.invoke(query)
    context = ("\n").join(doc.page_content for doc in docs)
    return context

custom_rag_chain = (
    {"context": RunnableLambda(format_docs), "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Usage:
response = custom_rag_chain.invoke("what is your return policy and how do i track my order?")

In [17]:
response

'Our return policy allows you to return items within 30 days. To track your order, you can use the tracking link that was sent to your email.'